# G2++ Model

As a final example, let us look at a calibration example of the 2-factor G2++ model.

$drt = \phi(t)+ x_t+ y_t$

where $x_t$ and $y_t$ are defined by

$dx_t = −a x_t dt + \sigma dW^1_t$

$dy_t = −b y_t dt + \eta dW^2_t$

$⟨dW^1_t dW^2_t⟩= \rho dt$

Once again, we use the TreeSwaptionEngine to value the swaptions in the calibration step.

In [1]:
import QuantLib as ql
from collections import namedtuple
import math
import calibration_helper

In [2]:
today = ql.Date(15, ql.February, 2002);
settlement= ql.Date(19, ql.February, 2002);
ql.Settings.instance().evaluationDate = today;
term_structure = ql.YieldTermStructureHandle(ql.FlatForward(settlement,0.04875825,ql.Actual365Fixed()))
index = ql.Euribor1Y(term_structure)

In [3]:
CalibrationData = namedtuple("CalibrationData", 
                             "start, length, volatility")
data = [CalibrationData(1, 5, 0.1148),
        CalibrationData(2, 4, 0.1108),
        CalibrationData(3, 3, 0.1070),
        CalibrationData(4, 2, 0.1021),
        CalibrationData(5, 1, 0.1000 )]

In [4]:
model = ql.G2(term_structure)
engine = ql.TreeSwaptionEngine(model, 25)
# engine = ql.G2SwaptionEngine(model, 10, 400)
# engine = ql.FdG2SwaptionEngine(model)

swaptions = calibration_helper.create_swaption_helpers(data, index, term_structure, engine)

optimization_method = ql.LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = ql.EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)

a, sigma, b, eta, rho = model.params()
print("a = %6.5f, sigma = %6.5f, b = %6.5f, eta = %6.5f, rho = %6.5f " % (a, sigma, b, eta, rho))


a = 0.04511, sigma = 0.00301, b = 0.04041, eta = 0.00473, rho = 0.03500 


In [5]:
calibration_helper.calibration_report(swaptions, data)

----------------------------------------------------------------------------------
    Model Price    Market Price     Implied Vol      Market Vol       Rel Error
----------------------------------------------------------------------------------
        0.00869         0.00949         0.10515         0.11480        -0.08394
        0.00967         0.01008         0.10631         0.11080        -0.04043
        0.00867         0.00872         0.10640         0.10700        -0.00563
        0.00650         0.00623         0.10662         0.10210         0.04414
        0.00356         0.00332         0.10705         0.10000         0.07013
----------------------------------------------------------------------------------
Cumulative Error :         0.12514
